### Librerias 

In [1]:
import nltk
import numpy as np
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import re



In [3]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
nltk.download('stopwords')
tokenizer = TweetTokenizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Función de preprocesar

In [5]:
def count_words(tweet, words):
    word_count = {}
    tokens = tweet.split()  # Assuming your 'Tokens' field or similar tokenization
    for word in words:
        word_count[word] = tokens.count(word)
    return pd.Series(word_count)

In [6]:
from nltk.tokenize import TweetTokenizer #importamos el tokenizador
tweet_token= TweetTokenizer()

In [7]:
def preprocesar(text):  #Obtenemos tokens el texto

    text = re.sub(r'https?://\S+|www\.\S+', '', text) 
    text = text.lower()  # Convertimos en minusculas
    text = ''.join([c for c in text if c.isalnum() or c.isspace()])  # quitamos puntación
    
    # Tokenization
    tokens = tweet_token.tokenize(text)
    #stopwords_list = stopwords.words('spanish')
    
    #filtered_tokens = [word for word in tokens if word not in stopwords_list]  # Usamos los tokens filatrados
    
    # Lematizacion
    #doc = nlp(" ".join(filtered_tokens))  #usamos solo los tokens filtrados
    #lemmas = [token.lemma_ for token in doc]
    
    return " ".join(tokens)


# LSA Debate

### Separar participacion función 

In [8]:
def extract_speech_from_file_improved(file_path, target_speaker, other_speakers):
    
    collected_text = []
    capture = False
    found_target_speaker = False

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            normalized_line = line.strip()

            if target_speaker in normalized_line:
                found_target_speaker = True
            
            if any(speaker in normalized_line for speaker in other_speakers) and not target_speaker in normalized_line:
                if capture:
                    capture = False

            # Empezar a agreagar hasta que el empiece el candidato objetivo
            if found_target_speaker and normalized_line and not any(speaker in normalized_line for speaker in other_speakers):
                capture = True
                found_target_speaker = False  # Reiniciar para que solo el texto subsecuente sea utilizado

            if capture:
                collected_text.append(normalized_line)

    return "\n".join(collected_text)



## Claudia

### Claudia : Intervenciones

In [9]:
result_clau = extract_speech_from_file_improved("D:\PythonCode\LN\8tavo\Hackaton\Debate.txt",
                                   "Candidata a la Presidencia por la Coalición “Sigamos Haciendo Historia”, Claudia Sheinbaum Pardo",
                                   ["Candidata a la Presidencia por la Coalición “Fuerza y Corazón por México”, Bertha Xóchitl Gálvez Ruiz",
                                    "Candidato a la Presidencia del Partido Movimiento Ciudadano, Jorge Álvarez Máynez",
                                    "Moderador,",
                                    "Moderadora,"])

In [10]:
result_list_clau = result_clau.split("\n\n")

In [11]:
phrase_to_remove = "Candidata a la Presidencia por la Coalición “Sigamos Haciendo Historia”, Claudia Sheinbaum Pardo:"

cleaned_result_list_clau = [element.replace(phrase_to_remove, "") for element in result_list_clau]


In [12]:
for i in range(len(cleaned_result_list_clau)):
    cleaned_result_list_clau[i] = preprocesar(cleaned_result_list_clau[i])

In [13]:
cleaned_result_list_clau = np.array(cleaned_result_list_clau)

In [14]:
import csv

with open('claudia.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for element in cleaned_result_list_clau:
        writer.writerow([element])  

print("CSV file has been created successfully.")


CSV file has been created successfully.


### LSA Claudia

In [15]:
stopwords_lista = stopwords.words('spanish')

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

tfidf_vectorizer = TfidfVectorizer(norm='l2', max_features=10000, stop_words = stopwords_lista)  #usamos la norma l2 para normalizar y usamos maximo 10000, sobre todos los docuementos
lsa_model = TruncatedSVD(n_components=5, random_state=42) # Creamos una instancia de una SVD truncada
#con 3 componentes principales que vamos a mantener despues de la transformación.

In [17]:
    
# Creamos la matriz TF-IDF para el lugar actual

tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_result_list_clau)
    
# Aplicamos LSA
lsa_matrix = lsa_model.fit_transform(tfidf_matrix)
    
# Extraemos las mejores palabras de cada topico

topic_words = []
for i, topic in enumerate(lsa_model.components_):
    top_feature_indices = abs(topic).argsort()[-10:][::-1]
    top_features = [tfidf_vectorizer.get_feature_names_out()[index] for index in top_feature_indices]
    topic_words.append((i+1, top_features))
    
topics = topic_words

In [18]:
print("Topicos más relevantes, Candidato A :")
for topic_number, words in topics:
        print(f"Topico {topic_number}: {', '.join(words)}")

Topicos más relevantes, Candidato A :
Topico 1: vamos, méxico, ciudad, temas, hicimos, salud, dos, hablar, voy, gobierno
Topico 2: vamos, temas, salud, hablar, voy, pública, méxico, plantear, sistema, ciudad
Topico 3: méxico, ciudad, sistema, salud, hicimos, pública, supuesto, imss, fortalecer, ley
Topico 4: cómo, prian, mentirosa, candidata, creerle, vamos, temas, candidato, hablar, gobierno
Topico 5: cómo, prian, candidata, temas, vamos, plantear, hablar, gobierno, dos, tiempo


## Xochitl

### Xochitl data

In [19]:
result_xoch = extract_speech_from_file_improved("D:\PythonCode\LN\8tavo\Hackaton\Debate.txt",
                                   "Candidata a la Presidencia por la Coalición “Fuerza y Corazón por México”, Bertha Xóchitl Gálvez Ruiz",
                                   ["Candidata a la Presidencia por la Coalición “Sigamos Haciendo Historia”, Claudia Sheinbaum Pardo",
                                    "Candidato a la Presidencia del Partido Movimiento Ciudadano, Jorge Álvarez Máynez",
                                    "Moderador,",
                                    "Moderadora,"])

In [20]:
result_list_xoch = result_xoch.split("\n\n")


In [21]:
phrase_to_remove = "Candidata a la Presidencia por la Coalición “Fuerza y Corazón por México”, Bertha Xóchitl Gálvez Ruiz:"

cleaned_result_list_xoch = [element.replace(phrase_to_remove, "") for element in result_list_xoch]

In [22]:
for i in range(len(cleaned_result_list_xoch)):
    cleaned_result_list_xoch[i] = preprocesar(cleaned_result_list_xoch[i])

In [23]:
cleaned_result_list_xoch = np.array(cleaned_result_list_xoch)

In [24]:
import csv


with open('xochitl.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for element in cleaned_result_list_xoch:
        writer.writerow([element]) 

print("CSV file has been created successfully.")


CSV file has been created successfully.


### Xochitl LSA

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

tfidf_vectorizer = TfidfVectorizer(norm='l2', max_features=10000, stop_words = stopwords_lista)  #usamos la norma l2 para normalizar y usamos maximo 10000, sobre todos los docuementos
lsa_model = TruncatedSVD(n_components=5, random_state=42) # Creamos una instancia de una SVD truncada
#con 3 componentes principales que vamos a mantener despues de la transformación.

In [26]:
    
# Creamos la matriz TF-IDF para el lugar actual

tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_result_list_xoch)
    
# Aplicamos LSA
lsa_matrix = lsa_model.fit_transform(tfidf_matrix)
    
# Extraemos las mejores palabras de cada topico

topic_words = []
for i, topic in enumerate(lsa_model.components_):
    top_feature_indices = abs(topic).argsort()[-10:][::-1]
    top_features = [tfidf_vectorizer.get_feature_names_out()[index] for index in top_feature_indices]
    topic_words.append((i+1, top_features))
    
topics = topic_words
print("Topicos más relevantes, Candidato B :")
for topic_number, words in topics:
        print(f"Topico {topic_number}: {', '.join(words)}")

Topicos más relevantes, Candidato B :
Topico 1: voy, mujeres, méxico, aquí, claudia, gobierno, corrupción, salud, mujer, va
Topico 2: aquí, supuesto, gobierno, mujeres, claudia, inai, colegio, va, mil, pues
Topico 3: supuesto, voy, claudia, colegio, hiciste, bueno, castigar, sometería, sociales, todas
Topico 4: supuesto, voy, méxico, sometería, salud, educación, mujer, gobierno, presidenta, aquí
Topico 5: mujeres, beca, mujer, ningún, voy, sociales, universal, programas, partido, realmente


## Maynez

### Maynez

In [27]:
result_may = extract_speech_from_file_improved("D:\PythonCode\LN\8tavo\Hackaton\Debate.txt",
                                   "Candidato a la Presidencia del Partido Movimiento Ciudadano, Jorge Álvarez Máynez",
                                   ["Candidata a la Presidencia por la Coalición “Sigamos Haciendo Historia”, Claudia Sheinbaum Pardo",
                                    "Candidata a la Presidencia por la Coalición “Fuerza y Corazón por México”, Bertha Xóchitl Gálvez Ruiz",
                                    "Moderador,",
                                    "Moderadora,"])

In [28]:
result_list_may = result_may.split("\n\n")


In [29]:
phrase_to_remove = "Candidato a la Presidencia del Partido Movimiento Ciudadano, Jorge Álvarez Máynez:"

cleaned_result_list_may = [element.replace(phrase_to_remove, "") for element in result_list_may]

In [30]:
for i in range(len(cleaned_result_list_may)):
    cleaned_result_list_may[i] = preprocesar(cleaned_result_list_may[i])

In [31]:
cleaned_result_list_may = np.array(cleaned_result_list_may)

In [32]:
import csv


with open('maynez.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    for element in cleaned_result_list_may:
        writer.writerow([element]) 

print("CSV file has been created successfully.")


CSV file has been created successfully.


### Maynez LSA

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

tfidf_vectorizer = TfidfVectorizer(norm='l2', max_features=10000, stop_words = stopwords_lista)  #usamos la norma l2 para normalizar y usamos maximo 10000, sobre todos los docuementos
lsa_model = TruncatedSVD(n_components=5, random_state=42) # Creamos una instancia de una SVD truncada
#con 3 componentes principales que vamos a mantener despues de la transformación.

In [46]:
    
# Creamos la matriz TF-IDF para el lugar actual

tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_result_list_may)
    
# Aplicamos LSA
lsa_matrix = lsa_model.fit_transform(tfidf_matrix)
    
# Extraemos las mejores palabras de cada topico

topic_words = []
for i, topic in enumerate(lsa_model.components_):
    top_feature_indices = abs(topic).argsort()[-10:][::-1]
    top_features = [tfidf_vectorizer.get_feature_names_out()[index] for index in top_feature_indices]
    topic_words.append((i+1, top_features))
    
topics = topic_words
print("Topicos más relevantes, Candidato C :")

for topic_number, words in topics:
        print(f"Topico {topic_number}: {', '.join(words)}")

Topicos más relevantes, Candidato C :
Topico 1: sistema, méxico, vamos, personas, ser, presidente, política, tema, nuevo, corrupción
Topico 2: personas, todas, derechos, modelo, creo, nuevo, sistema, supuesto, corrupción, política
Topico 3: niños, millones, niñas, gobierno, pesos, mil, vieja, quiero, política, votan
Topico 4: vamos, nuevo, méxico, hijos, todas, derechos, juntos, creo, gobierno, tema
Topico 5: bueno, millones, pesos, moral, autoridad, tener, cinco, corrupción, mil, gobierno
